# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 10.3 MB/s eta 0:00:00


In [3]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Load trainset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [5]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [6]:
feature_list = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

In [7]:
fgsm_path = '/content/drive/MyDrive/Dataset/insdn/adv/24_feature/fgm/fgm_mlp_bin_feature.csv'
#fgsm_label_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/fgsm/adv_fgsm_target.csv'

In [8]:
fgsm_df = pd.read_csv(fgsm_path)

In [9]:
fgsm_df.columns = feature_list
fgsm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      55151 non-null  float64
 1   Tot Fwd Pkts       55151 non-null  float64
 2   Tot Bwd Pkts       55151 non-null  float64
 3   TotLen Fwd Pkts    55151 non-null  float64
 4   TotLen Bwd Pkts    55151 non-null  float64
 5   Fwd Pkt Len Max    55151 non-null  float64
 6   Fwd Pkt Len Min    55151 non-null  float64
 7   Fwd Pkt Len Mean   55151 non-null  float64
 8   Fwd Pkt Len Std    55151 non-null  float64
 9   Bwd Pkt Len Max    55151 non-null  float64
 10  Bwd Pkt Len Min    55151 non-null  float64
 11  Bwd Pkt Len Mean   55151 non-null  float64
 12  Bwd Pkt Len Std    55151 non-null  float64
 13  Flow Byts/s        55151 non-null  float64
 14  Flow Pkts/s        55151 non-null  float64
 15  Flow IAT Mean      55151 non-null  float64
 16  Flow IAT Std       551

In [ ]:
#fgsm_label = pd.read_csv(fgsm_label_path)
#fgsm_label.info()

# Load DL-based IDS model

In [10]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

# SHAP

In [11]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100, random_state = 42))

In [12]:
fgsm_1000 = fgsm_df.head(1000)

In [13]:
shap_values = explainer.shap_values(fgsm_1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
data_df = pd.DataFrame(shap_values[0])

In [15]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.000000,0.000000,0.000000,0.000000,0.024031,0.0,0.000000,-0.006697,0.014128,0.000000,...,0.000000,0.0,-0.020124,0.006892,0.000000,-0.005990,0.012941,0.000000,0.000000,-0.006614
1,0.000000,0.000000,0.006078,0.000000,0.006404,0.0,0.000000,-0.004423,0.000000,0.000000,...,0.011177,0.0,-0.010147,0.000000,-0.013520,0.003079,-0.052375,0.000000,-0.045104,-0.060366
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.007689,0.000000,0.000000,0.000000,...,0.015416,0.0,0.000000,0.000000,-0.012073,0.000000,-0.053471,0.000000,-0.029122,-0.057386
3,0.012168,0.000000,0.000000,0.012676,0.000000,0.0,0.000000,-0.006325,0.004575,0.000000,...,0.000000,0.0,0.000000,0.000000,0.008074,-0.009457,-0.064633,0.000000,-0.033876,-0.054501
4,-0.068860,-0.015702,-0.011771,0.015973,0.000000,0.0,-0.009986,0.006199,0.010662,-0.004604,...,-0.007417,0.0,0.080588,-0.211684,0.052976,-0.017468,-0.169244,-0.014491,-0.148827,-0.112607


In [ ]:
data_df.to_csv('/content/drive/MyDrive/1000_samples_bin_[0].csv', index=False)

In [16]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000000,0.000000,0.000000,0.000000,0.024031,0.0,0.000000,-0.006697,0.014128,0.000000,...,0.000000,0.0,-0.020124,0.006892,0.000000,-0.005990,0.012941,0.000000,0.000000,-0.006614
1,0.000000,0.000000,0.006078,0.000000,0.006404,0.0,0.000000,-0.004423,0.000000,0.000000,...,0.011177,0.0,-0.010147,0.000000,-0.013520,0.003079,-0.052375,0.000000,-0.045104,-0.060366
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.007689,0.000000,0.000000,0.000000,...,0.015416,0.0,0.000000,0.000000,-0.012073,0.000000,-0.053471,0.000000,-0.029122,-0.057386
3,0.012168,0.000000,0.000000,0.012676,0.000000,0.0,0.000000,-0.006325,0.004575,0.000000,...,0.000000,0.0,0.000000,0.000000,0.008074,-0.009457,-0.064633,0.000000,-0.033876,-0.054501
4,-0.068860,-0.015702,-0.011771,0.015973,0.000000,0.0,-0.009986,0.006199,0.010662,-0.004604,...,-0.007417,0.0,0.080588,-0.211684,0.052976,-0.017468,-0.169244,-0.014491,-0.148827,-0.112607


In [17]:
data_dict = {}
for idx in range(0, 1000):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [18]:
print(data_dict)

{'Adv sample 0': ['Bwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Max', 'Fwd Pkts/b Avg', 'Init Bwd Win Byts', 'Bwd Pkts/s', 'Bwd Pkt Len Std', 'TotLen Bwd Pkts', 'Fwd Seg Size Avg', 'Pkt Len Mean'], 'Adv sample 1': ['SYN Flag Cnt', 'Bwd IAT Mean', 'Fwd IAT Min', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Fwd Byts/b Avg', 'Bwd Pkts/s', 'Fwd Header Len', 'Fwd IAT Mean', 'Flow IAT Max'], 'Adv sample 2': ['SYN Flag Cnt', 'Bwd IAT Mean', 'Init Bwd Win Byts', 'Fwd IAT Min', 'Bwd IAT Max', 'Bwd Pkts/b Avg', 'Bwd IAT Tot', 'Fwd IAT Mean', 'Bwd Seg Size Avg', 'Pkt Len Max'], 'Adv sample 3': ['SYN Flag Cnt', 'Bwd IAT Mean', 'Fwd IAT Min', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Fwd Byts/b Avg', 'Bwd Pkts/s', 'Bwd Pkts/b Avg', 'Flow IAT Min', 'Pkt Len Max'], 'Adv sample 4': ['ACK Flag Cnt', 'Active Mean', 'Active Max', 'Flow IAT Mean', 'Flow IAT Std', 'Init Bwd Win Byts', 'Bwd Pkt Len Min', 'TotLen Fwd Pkts', 'Bwd PSH Flags', 'Fwd Pkts/s'], 'Adv sample 5': ['Bwd IAT Mean', 'Bwd IAT Min', 'Fwd Pkts/b Avg', 'Init 

In [19]:
len(shap_values[0])

1000

In [20]:
len(data_dict)

1000

# Detection phase

In [21]:
import os
import pandas as pd

In [27]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/MLP_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,Init Bwd Win Byts,786
1,Idle Max,546
2,Bwd Pkt Len Max,531
3,Idle Mean,449
4,Tot Fwd Pkts,432
5,Idle Min,413
6,Bwd Pkt Len Std,399
7,Tot Bwd Pkts,395
8,Bwd Pkt Len Mean,382
9,TotLen Fwd Pkts,373


## Detection rate (top 51) = 997/1000

In [34]:
whitelist = whitelist_df.iloc[:, 0][:51].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Tot Fwd Pkts', 'Idle Min', 'Bwd Pkt Len Std', 'Tot Bwd Pkts', 'Bwd Pkt Len Mean', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Fwd IAT Max', 'Bwd Seg Size Avg', 'Pkt Len Min', 'SYN Flag Cnt', 'Flow Duration', 'Fwd Pkt Len Std', 'Fwd IAT Tot', 'Bwd IAT Std', 'Flow Pkts/s', 'Down/Up Ratio', 'Fwd IAT Std', 'Fwd Pkt Len Min', 'Fwd Pkt Len Max', 'Subflow Bwd Pkts', 'Bwd Pkt Len Min', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd Header Len', 'Subflow Bwd Byts', 'Subflow Fwd Byts', 'Fwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Fwd Pkts/s', 'Fwd Act Data Pkts', 'Pkt Len Max', 'Fwd IAT Mean', 'Active Max', 'Active Std', 'Pkt Len Std', 'Pkt Len Var', 'Subflow Fwd Pkts', 'Flow Byts/s', 'Pkt Size Avg', 'Active Mean']


In [35]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:997


## Detection rate (top 59) = 902/1000

In [30]:
whitelist2 = whitelist_df.iloc[:, 0][:59].tolist()
print(whitelist2)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Tot Fwd Pkts', 'Idle Min', 'Bwd Pkt Len Std', 'Tot Bwd Pkts', 'Bwd Pkt Len Mean', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Fwd IAT Max', 'Bwd Seg Size Avg', 'Pkt Len Min', 'SYN Flag Cnt', 'Flow Duration', 'Fwd Pkt Len Std', 'Fwd IAT Tot', 'Bwd IAT Std', 'Flow Pkts/s', 'Down/Up Ratio', 'Fwd IAT Std', 'Fwd Pkt Len Min', 'Fwd Pkt Len Max', 'Subflow Bwd Pkts', 'Bwd Pkt Len Min', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd Header Len', 'Subflow Bwd Byts', 'Subflow Fwd Byts', 'Fwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Fwd Pkts/s', 'Fwd Act Data Pkts', 'Pkt Len Max', 'Fwd IAT Mean', 'Active Max', 'Active Std', 'Pkt Len Std', 'Pkt Len Var', 'Subflow Fwd Pkts', 'Flow Byts/s', 'Pkt Size Avg', 'Active Mean', 'ACK Flag Cnt', 'Flow IAT Max', 'Active Min', 'Pkt Len Mean', 'Fwd Pkt Len Mean', 'Flow IAT Std', 'Fwd Seg Size Avg', 'PSH Flag Cnt']


In [31]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:902
